In [1]:
#################################
# filenames
#################################

number = 43

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(11)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(11)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    #time.sleep(1)
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(11)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(11)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(11)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(11)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo[:21])
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615685451.6052768
500
1/500 1,4,1,L46
histo1 1
2/500 67,17,4,16
histo1 2
3/500 27,57/6,4,5
histo1 1
4/500 4,5/6,4,2
histo1 1
5/500 4,b6,1,5/5
histo1 1
6/500 27,2,27,D2
histo1 1
7/500 1,164,1,164
histo1 35
8/500 b4,1,b6,b4
histo1 1
9/500 1,5,6,56
histo1 13
10/500 b7,1,564/6,5/6
histo1 1
11/500 27,1,57/5,57
histo1 1
12/500 2,3,2,6
histo1 2
13/500 4,L4,4,5/5
histo1 1
14/500 1,5,b6,5/2
histo1 3
15/500 17,5/2,4,7/5
histo1 1
16/500 4,5,b3,4
histo1 4
17/500 6,2,6,b4
histo1 1
18/500 5/5,2,4,5
histo1 1
19/500 542,27,b7,Y564
histo1 1
20/500 164,16,2,4
histo1 1
21/500 57,1,67,57/5
histo1 1
22/500 56,1,47,56
histo1 1
23/500 4,16,2,Y2
histo1 1
24/500 1,b3,4,5
histo1 6
25/500 36,265,36,6
histo1 1
26/500 5/6,4,5,7/6
histo1 1
27/500 46,1,5,1
histo1 3
28/500 5,164,1,5
histo1 2
29/500 5,1,142,M142
histo1 1
30/500 4,5,5/2,5/5
histo1 1
31/500 5/5,5,665,57/5
histo1 1
32/500 6,2,5,2
histo1 3
33/500 2,1,565,1
histo1 1
34/500 37,6,57/5,27
histo1 1
35/500 4,b46,1,L265
histo1 1
36/500 56,243,36,465
histo1 1
37

histo1 1
292/500 57/5,37,565/5,5
histo1 1
293/500 2,5,6,5/6
histo1 1
294/500 6,17,342,47
histo1 1
295/500 46,b6,164,5
histo1 1
296/500 6,664,6,76
histo1 1
297/500 1,b3,5,57
histo1 1
298/500 5,46,164,46
histo1 1
299/500 4,5,6,57/6
histo1 3
300/500 4,5,46,164
histo1 1
301/500 2,1,4,6
histo1 8
302/500 47,57/6,67,27
histo1 1
303/500 b4,1,b5,4
histo1 1
304/500 56,6,1,5
histo1 2
305/500 66,M76,664,57/6
histo1 2
306/500 5/5,6,67,5/5
histo1 1
307/500 5,Y5,5,57
histo1 1
308/500 26,5/6,6,56/6
histo1 1
309/500 6,4,5/6,2
histo1 1
310/500 164,57,1,47
histo1 1
311/500 4,6,16,4
histo1 3
312/500 b4,b464,37,343
histo1 1
313/500 5/2,2,37,4
histo1 1
314/500 1,164,5,564
histo1 3
315/500 5,56/6,6,36
histo1 2
316/500 2,6,47,57/6
histo1 1
317/500 57,4,b4,1
histo1 1
318/500 47,1,5,164
histo1 2
319/500 5/5,5,1,5/5
histo1 3
320/500 1,4,16,464
histo1 1
321/500 26,2,37,6
histo1 1
322/500 3,27,5,57/6
histo1 1
323/500 17,5,17,b57
histo1 1
324/500 3,6,4,57
histo1 3
325/500 67,5,1,47
histo1 4
326/500 2,16,464,6
histo

In [ ]:
##########################
# list errors here
##########################
40/500 36,6,66,6
2. Unable to extract for 36 6 66 6 page 1
histo2 20
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

In [2]:
##########################
# test errors here
##########################
mykey = '36,6,66,6'
page = 1
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [3]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 1
# print(histo[:21])
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 378, 61, 20, 10, 8, 2, 2, 2, 4, 4, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0]
